# Imports

In [1]:
import itertools
import os
import typing

from extract import PrefixType
from utils import file_utils
from utils_extraction import load_utils

# Constants

In [2]:
DEFAULT_ENV_VARS = dict(
    MODEL="meta-llama/Llama-2-7b-chat-hf",
    DATASETS="dbpedia-14",
    # LABELED_DATASETS="imdb",
    EVAL_DATASETS="burns",
    PREFIX="normal",
    METHOD_LIST=None,  # Must be set.
    MODE="auto",
    LAYER=-1,
    LR=1e-2,
    N_EPOCHS=5000,
    WEIGHT_DECAY=0.0,
    OPT="sgd",
    NUM_SEEDS=1,
    N_TRIES=1,
    SPAN_DIRS_COMBINATION="convex",
    # Saving
    SAVE_PARAMS=False,
    SAVE_FIT_RESULT=True,
    SAVE_FIT_PLOTS=False,
    SAVE_STATES=False,
    SAVE_ORTHOGONAL_DIRECTIONS=False,
)

ALL_DATASETS = [
    "imdb",
    "amazon-polarity",
    "ag-news",
    "dbpedia-14",
    "copa",
    "rte",
    "boolq",
    "qnli",
    "piqa",
]
ALL_DATASET_PAIRS = list(itertools.product(ALL_DATASETS, ALL_DATASETS))

# Utils

In [3]:
MODEL_NAME_TO_TAG = {
    "meta-llama/Llama-2-7b-chat-hf": "llama-2-7b-chat-hf",
    "meta-llama/Llama-2-13b-chat-hf": "llama-2-13b-chat-hf",
    "meta-llama/Meta-Llama-3-8B": "meta-llama/Meta-Llama-3-8B",
    "mistralai/Mistral-7B-Instruct-v0.2": "mistralai/Mistral-7B-Instruct-v0.2",
    "meta-llama/Meta-Llama-3-8B-Instruct": "meta-llama/Meta-Llama-3-8B-Instruct",
}


def make_ccs_lr_sub_tag(env_vars: dict) -> str:
    mode = env_vars["MODE"]
    if mode == "auto":
        raise ValueError("Set MODE explicitly instead of using 'auto'.")

    tag = f"/ccs_lr/mode_{mode}/sup_weight_{env_vars['SUP_WEIGHT']}/unsup_weight_{env_vars['UNSUP_WEIGHT']}/lr_{env_vars['LR']}/n_epochs_{env_vars['N_EPOCHS']}"

    weight_decay = env_vars["WEIGHT_DECAY"]
    if weight_decay > 0:
        tag += f"/weight_decay_{weight_decay}"

    return tag


def make_pseudolabel_sub_tag(env_vars: dict) -> str:
    tag = f"/pseudolabel/rounds_{env_vars['PSEUDOLABEL_N_ROUNDS']}"
    select_fn = env_vars["PSEUDOLABEL_SELECT_FN"]
    tag += f"/select_{select_fn}"
    if select_fn == "high_confidence_consistency":
        tag += f"/prob_thres_{env_vars['PSEUDOLABEL_PROB_THRESHOLD']}"
    elif select_fn != "all":
        raise NotImplementedError(f"Unknown select_fn: {select_fn}")
    tag += f"/label_{env_vars['PSEUDOLABEL_LABEL_FN']}"
    tag += make_ccs_lr_sub_tag(env_vars)
    return tag


def make_tag(env_vars: dict) -> str:
    tag = ""

    # Model
    model = env_vars["MODEL"]
    if model not in MODEL_NAME_TO_TAG:
        raise ValueError(f"Unknown model: {model}")
    tag += MODEL_NAME_TO_TAG[model]

    # Prefix
    test_prefix = env_vars.get("TEST_PREFIX", None)
    if test_prefix is None:
        env_vars["TEST_PREFIX"] = env_vars["PREFIX"]
    elif env_vars["PREFIX"] != test_prefix:
        raise NotImplementedError("Different test prefix not supported")
    tag += f"/{env_vars['PREFIX']}"

    # Layer
    tag += f"/layer_{env_vars['LAYER']}"

    # Method
    method = env_vars["METHOD_LIST"]
    if isinstance(method, (list, tuple)):
        if len(method) > 1:
            raise ValueError("Only one method supported at a time.")
        method = method[0]
    if method == "pseudolabel":
        tag += make_pseudolabel_sub_tag(env_vars)
    elif method == "CCS+LR":
        tag += make_ccs_lr_sub_tag(env_vars)
    elif method == "CCS+LR-in-span":
        mode = env_vars["MODE"]
        if mode == "auto":
            raise ValueError("Set MODE explicitly instead of using 'auto'.")

        tag += f"/ccs_lr_in_span/mode_{mode}/sup_weight_{env_vars['SUP_WEIGHT']}/unsup_weight_{env_vars['UNSUP_WEIGHT']}/lr_{env_vars['LR']}/n_epochs_{env_vars['N_EPOCHS']}/n_orth_dirs_{env_vars['NUM_ORTHOGONAL_DIRECTIONS']}/span_dirs_combo_{env_vars['SPAN_DIRS_COMBINATION']}"
    else:
        raise NotImplementedError(f"Method {method} not supported.")

    return tag


def validate_env_vars(env_vars: dict) -> None:
    prefix = env_vars["PREFIX"]
    if prefix not in typing.get_args(PrefixType):
        raise ValueError(f"Unknown prefix: {prefix}")

    test_prefix = env_vars.get("TEST_PREFIX")
    if test_prefix is not None and test_prefix not in typing.get_args(PrefixType):
        raise ValueError(f"Unknown test prefix: {test_prefix}")

    if not env_vars.get("METHOD_LIST"):
        raise ValueError("METHOD_LIST must be set.")


def make_env_vars_for_experiment(experiment_config: dict) -> dict:
    env_vars = DEFAULT_ENV_VARS.copy()
    env_vars.update(experiment_config)
    env_vars["NAME"] = make_tag(env_vars)

    validate_env_vars(env_vars)
    return env_vars


MODEL_TO_SLURM_MEM = {
    "meta-llama/Llama-2-7b-chat-hf": 16,
    "meta-llama/Llama-2-13b-chat-hf": 16,
}


def make_slurm_args(env_vars: dict) -> str:
    args = []
    mem = MODEL_TO_SLURM_MEM.get(env_vars["MODEL"])
    if mem is not None:
        args.append(f"--mem={mem}gb")

    return " ".join(args)


def make_train_command_for_experiment(env_vars: dict, slurm: bool = True) -> str:
    env_vars_str = " ".join(f"{key}={value}" for key, value in sorted(env_vars.items()))
    if slurm:
        slurm_args = make_slurm_args(env_vars)
        cmd = f"sbatch {slurm_args} slurm_extract.sh"
    else:
        cmd = "./extract.sh"

    return f"{env_vars_str} {cmd}"


def print_train_commands_for_experiments_all_datasets(
    experiment_configs: list[dict], slurm: bool = True
):
    env_vars_list = []
    for experiment_config in experiment_configs:
        for ds in ALL_DATASETS:
            ds_experiment_config = experiment_config.copy()
            if "DATASETS" in ds_experiment_config:
                raise ValueError("DATASETS should not be set in ds_experiment_config.")
            if "LABELED_DATASETS" in ds_experiment_config:
                raise ValueError(
                    "LABELED_DATASETS should not be set in ds_experiment_config."
                )
            if "EVAL_DATASETS" in ds_experiment_config:
                raise ValueError(
                    "EVAL_DATASETS should not be set in ds_experiment_config."
                )

            ds_experiment_config["DATASETS"] = ds
            ds_experiment_config["EVAL_DATASETS"] = "burns"
            env_vars_list.append(make_env_vars_for_experiment(ds_experiment_config))

    for env_vars in env_vars_list:
        print(make_train_command_for_experiment(env_vars, slurm=slurm), end="\n\n")


def print_train_commands_for_experiments_all_dataset_pairs(
    experiment_configs: list[dict], slurm: bool = True
):
    env_vars_list = []
    for experiment_config in experiment_configs:
        # Iterate over all dataset pairs.
        for labeled_ds, unlabeled_ds in ALL_DATASET_PAIRS:
            ds_experiment_config = experiment_config.copy()
            if "DATASETS" in ds_experiment_config:
                raise ValueError("DATASETS should not be set in ds_experiment_config.")
            if "LABELED_DATASETS" in ds_experiment_config:
                raise ValueError(
                    "LABELED_DATASETS should not be set in ds_experiment_config."
                )
            if "EVAL_DATASETS" in ds_experiment_config:
                raise ValueError(
                    "EVAL_DATASETS should not be set in ds_experiment_config."
                )

            ds_experiment_config["DATASETS"] = unlabeled_ds
            ds_experiment_config["LABELED_DATASETS"] = labeled_ds
            ds_experiment_config["EVAL_DATASETS"] = (
                f'"{list(set([labeled_ds, unlabeled_ds]))}"'
            )
            env_vars_list.append(make_env_vars_for_experiment(ds_experiment_config))

    for env_vars in env_vars_list:
        print(make_train_command_for_experiment(env_vars, slurm=slurm), end="\n\n")

# Pseudo-label + LR

In [4]:
DEFAULT_PSEUDOLABEL_ENV_VARS = dict(
    METHOD_LIST="pseudolabel",
    MODE="concat",
    SUP_WEIGHT=1,
    UNSUP_WEIGHT=0,
    LR=1e-2,
    N_EPOCHS=5000,
    # Pseudolabel
    PSEUDOLABEL_N_ROUNDS=5,
    PSEUDOLABEL_SELECT_FN="high_confidence_consistency",
    PSEUDOLABEL_PROB_THRESHOLD=0.7,
    PSEUDOLABEL_LABEL_FN="argmax",
)

## select_fn=high_confidence_consistency label_fn=argmax

In [5]:
prefixes = ["normal-bananashed"]
models = [
    "meta-llama/Meta-Llama-3-8B-Instruct",
    # "meta-llama/Llama-2-13b-chat-hf",
    # "mistralai/Mistral-7B-Instruct-v0.2",
    # "meta-llama/Llama-2-7b-chat-hf",
]
layers = [-1]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(DEFAULT_PSEUDOLABEL_ENV_VARS, MODEL=model, LAYER=layer, PREFIX=prefix)
    )


# experiment_configs for all dataset pairs
print_train_commands_for_experiments_all_dataset_pairs(experiment_configs, slurm=True)

DATASETS=imdb EVAL_DATASETS="['imdb']" LABELED_DATASETS=imdb LAYER=-1 LR=0.01 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Meta-Llama-3-8B-Instruct NAME=meta-llama/Meta-Llama-3-8B-Instruct/normal-bananashed/layer_-1/pseudolabel/rounds_5/select_high_confidence_consistency/prob_thres_0.7/label_argmax/ccs_lr/mode_concat/sup_weight_1/unsup_weight_0/lr_0.01/n_epochs_5000 NUM_SEEDS=1 N_EPOCHS=5000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed PSEUDOLABEL_LABEL_FN=argmax PSEUDOLABEL_N_ROUNDS=5 PSEUDOLABEL_PROB_THRESHOLD=0.7 PSEUDOLABEL_SELECT_FN=high_confidence_consistency SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTIONS=False SAVE_PARAMS=False SAVE_STATES=False SPAN_DIRS_COMBINATION=convex SUP_WEIGHT=1 TEST_PREFIX=normal-bananashed UNSUP_WEIGHT=0 WEIGHT_DECAY=0.0 sbatch  slurm_extract.sh

DATASETS=amazon-polarity EVAL_DATASETS="['imdb', 'amazon-polarity']" LABELED_DATASETS=imdb LAYER=-1 LR=0.01 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Meta-Llama-3-8B-Inst

In [6]:
# # Only for experiment_configs
# env_vars_list = []
# for experiment_config in experiment_configs:
#     env_vars_list.append(make_env_vars_for_experiment(experiment_config))

# for env_vars in env_vars_list:
#     print(make_train_command_for_experiment(env_vars), end="\n\n")

## select_fn=all label_fn=argmax

In [7]:
DEFAULT_PSEUDOLABEL_SELECT_ALL_LABEL_ARGMAX_ENV_VARS = (
    DEFAULT_PSEUDOLABEL_ENV_VARS.copy()
)
DEFAULT_PSEUDOLABEL_SELECT_ALL_LABEL_ARGMAX_ENV_VARS.update(
    PSEUDOLABEL_N_ROUNDS=1, PSEUDOLABEL_SELECT_FN="all"
)

prefixes = ["bananashed"]
models = [
    "meta-llama/Llama-2-13b-chat-hf",
    "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "meta-llama/Llama-2-7b-chat-hf",
]
layers = [-1, -5, -9]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(
            DEFAULT_PSEUDOLABEL_SELECT_ALL_LABEL_ARGMAX_ENV_VARS,
            MODEL=model,
            LAYER=layer,
            PREFIX=prefix,
        )
    )

print_train_commands_for_experiments_all_dataset_pairs(experiment_configs)

DATASETS=imdb EVAL_DATASETS="['imdb']" LABELED_DATASETS=imdb LAYER=-1 LR=0.01 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/bananashed/layer_-1/pseudolabel/rounds_1/select_all/label_argmax/ccs_lr/mode_concat/sup_weight_1/unsup_weight_0/lr_0.01/n_epochs_5000 NUM_SEEDS=1 N_EPOCHS=5000 N_TRIES=1 OPT=sgd PREFIX=bananashed PSEUDOLABEL_LABEL_FN=argmax PSEUDOLABEL_N_ROUNDS=1 PSEUDOLABEL_PROB_THRESHOLD=0.7 PSEUDOLABEL_SELECT_FN=all SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTIONS=False SAVE_PARAMS=False SAVE_STATES=False SPAN_DIRS_COMBINATION=convex SUP_WEIGHT=1 TEST_PREFIX=bananashed UNSUP_WEIGHT=0 WEIGHT_DECAY=0.0 sbatch --mem=16gb slurm_extract.sh

DATASETS=amazon-polarity EVAL_DATASETS="['imdb', 'amazon-polarity']" LABELED_DATASETS=imdb LAYER=-1 LR=0.01 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/bananashed/layer_-1/pseudolabel/rounds_1/select_all/label_argmax/ccs_l

# Pseudo-label + CCS+LR

In [8]:
DEFAULT_PSEUDOLABEL_CCS_LR_ENV_VARS = dict(
    METHOD_LIST="pseudolabel",
    MODE="concat",
    SUP_WEIGHT=10,
    UNSUP_WEIGHT=1,
    LR=1e-3,
    N_EPOCHS=10000,
    # Pseudolabel
    PSEUDOLABEL_N_ROUNDS=5,
    PSEUDOLABEL_SELECT_FN="high_confidence_consistency",
    PSEUDOLABEL_PROB_THRESHOLD=0.7,
    PSEUDOLABEL_LABEL_FN="argmax",
)

## select_fn=high_confidence_consistency label_fn=argmax

In [9]:
prefixes = ["normal-bananashed"]
models = [
    "meta-llama/Meta-Llama-3-8B-Instruct",
    # "meta-llama/Llama-2-13b-chat-hf",
    # "mistralai/Mistral-7B-Instruct-v0.2",
    # "meta-llama/Llama-2-7b-chat-hf",
]
layers = [-1]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(
            DEFAULT_PSEUDOLABEL_CCS_LR_ENV_VARS, MODEL=model, LAYER=layer, PREFIX=prefix
        )
    )

print_train_commands_for_experiments_all_dataset_pairs(experiment_configs, slurm=True)

DATASETS=imdb EVAL_DATASETS="['imdb']" LABELED_DATASETS=imdb LAYER=-1 LR=0.001 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Meta-Llama-3-8B-Instruct NAME=meta-llama/Meta-Llama-3-8B-Instruct/normal-bananashed/layer_-1/pseudolabel/rounds_5/select_high_confidence_consistency/prob_thres_0.7/label_argmax/ccs_lr/mode_concat/sup_weight_10/unsup_weight_1/lr_0.001/n_epochs_10000 NUM_SEEDS=1 N_EPOCHS=10000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed PSEUDOLABEL_LABEL_FN=argmax PSEUDOLABEL_N_ROUNDS=5 PSEUDOLABEL_PROB_THRESHOLD=0.7 PSEUDOLABEL_SELECT_FN=high_confidence_consistency SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTIONS=False SAVE_PARAMS=False SAVE_STATES=False SPAN_DIRS_COMBINATION=convex SUP_WEIGHT=10 TEST_PREFIX=normal-bananashed UNSUP_WEIGHT=1 WEIGHT_DECAY=0.0 sbatch  slurm_extract.sh

DATASETS=amazon-polarity EVAL_DATASETS="['imdb', 'amazon-polarity']" LABELED_DATASETS=imdb LAYER=-1 LR=0.001 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Meta-Llama-3-

In [10]:
# # Only for experiment_configs
# env_vars_list = []
# for experiment_config in experiment_configs:
#     env_vars_list.append(make_env_vars_for_experiment(experiment_config))

# for env_vars in env_vars_list:
#     print(make_train_command_for_experiment(env_vars), end="\n\n")

## select_fn=all label_fn=argmax

In [11]:
DEFAULT_PSEUDOLABEL_CCS_LR_SELECT_ALL_LABEL_ARGMAX_ENV_VARS = (
    DEFAULT_PSEUDOLABEL_CCS_LR_ENV_VARS.copy()
)
DEFAULT_PSEUDOLABEL_CCS_LR_SELECT_ALL_LABEL_ARGMAX_ENV_VARS.update(
    PSEUDOLABEL_N_ROUNDS=1, PSEUDOLABEL_SELECT_FN="all"
)

prefixes = ["bananashed"]
models = [
    "meta-llama/Llama-2-13b-chat-hf",
    "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "meta-llama/Llama-2-7b-chat-hf",
]
layers = [-1, -5, -9]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(
            DEFAULT_PSEUDOLABEL_CCS_LR_SELECT_ALL_LABEL_ARGMAX_ENV_VARS,
            MODEL=model,
            LAYER=layer,
            PREFIX=prefix,
        )
    )

print_train_commands_for_experiments_all_dataset_pairs(experiment_configs, slurm=True)

DATASETS=imdb EVAL_DATASETS="['imdb']" LABELED_DATASETS=imdb LAYER=-1 LR=0.001 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/bananashed/layer_-1/pseudolabel/rounds_1/select_all/label_argmax/ccs_lr/mode_concat/sup_weight_10/unsup_weight_1/lr_0.001/n_epochs_10000 NUM_SEEDS=1 N_EPOCHS=10000 N_TRIES=1 OPT=sgd PREFIX=bananashed PSEUDOLABEL_LABEL_FN=argmax PSEUDOLABEL_N_ROUNDS=1 PSEUDOLABEL_PROB_THRESHOLD=0.7 PSEUDOLABEL_SELECT_FN=all SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTIONS=False SAVE_PARAMS=False SAVE_STATES=False SPAN_DIRS_COMBINATION=convex SUP_WEIGHT=10 TEST_PREFIX=bananashed UNSUP_WEIGHT=1 WEIGHT_DECAY=0.0 sbatch --mem=16gb slurm_extract.sh

DATASETS=amazon-polarity EVAL_DATASETS="['imdb', 'amazon-polarity']" LABELED_DATASETS=imdb LAYER=-1 LR=0.001 METHOD_LIST=pseudolabel MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/bananashed/layer_-1/pseudolabel/rounds_1/select_all/label_argma

# LR (CCS+LR impl)

In [12]:
DEFAULT_LR_ENV_VARS = dict(
    METHOD_LIST="CCS+LR",
    MODE="concat",
    SUP_WEIGHT=1,
    UNSUP_WEIGHT=0,
    LR=1e-2,
    N_EPOCHS=5000,
)

prefixes = ["normal-bananashed"]
models = [
    "meta-llama/Llama-2-13b-chat-hf",
    # "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "meta-llama/Llama-2-7b-chat-hf",
]
layers = [-1]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(DEFAULT_LR_ENV_VARS, MODEL=model, LAYER=layer, PREFIX=prefix)
    )

In [14]:
slurm = True

env_vars_list = []
for experiment_config in experiment_configs:
    for ds in ALL_DATASETS:
        ds_experiment_config = experiment_config.copy()
        if "DATASETS" in ds_experiment_config:
            raise ValueError("DATASETS should not be set in ds_experiment_config.")
        if "LABELED_DATASETS" in ds_experiment_config:
            raise ValueError(
                "LABELED_DATASETS should not be set in ds_experiment_config."
            )
        if "EVAL_DATASETS" in ds_experiment_config:
            raise ValueError("EVAL_DATASETS should not be set in ds_experiment_config.")

        ds_experiment_config["DATASETS"] = ds
        ds_experiment_config["LABELED_DATASETS"] = ds
        ds_experiment_config["EVAL_DATASETS"] = "burns"
        env_vars_list.append(make_env_vars_for_experiment(ds_experiment_config))

for env_vars in env_vars_list:
    print(make_train_command_for_experiment(env_vars, slurm=slurm), end="\n\n")

DATASETS=imdb EVAL_DATASETS=burns LABELED_DATASETS=imdb LAYER=-1 LR=0.01 METHOD_LIST=CCS+LR MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/normal-bananashed/layer_-1/ccs_lr/mode_concat/sup_weight_1/unsup_weight_0/lr_0.01/n_epochs_5000 NUM_SEEDS=1 N_EPOCHS=5000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTIONS=False SAVE_PARAMS=False SAVE_STATES=False SPAN_DIRS_COMBINATION=convex SUP_WEIGHT=1 TEST_PREFIX=normal-bananashed UNSUP_WEIGHT=0 WEIGHT_DECAY=0.0 sbatch --mem=16gb slurm_extract.sh

DATASETS=amazon-polarity EVAL_DATASETS=burns LABELED_DATASETS=amazon-polarity LAYER=-1 LR=0.01 METHOD_LIST=CCS+LR MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/normal-bananashed/layer_-1/ccs_lr/mode_concat/sup_weight_1/unsup_weight_0/lr_0.01/n_epochs_5000 NUM_SEEDS=1 N_EPOCHS=5000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTI

# CCS (CCS+LR impl)

In [15]:
DEFAULT_CCS_ENV_VARS = dict(
    METHOD_LIST="CCS+LR",
    MODE="concat",
    SUP_WEIGHT=0,
    UNSUP_WEIGHT=1,
    LR=1e-2,
    N_EPOCHS=5000,
)

prefixes = ["normal-bananashed"]
models = [
    "meta-llama/Llama-2-13b-chat-hf",
    # "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "meta-llama/Llama-2-7b-chat-hf",
]
layers = [-1]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(DEFAULT_CCS_ENV_VARS, MODEL=model, LAYER=layer, PREFIX=prefix)
    )

In [16]:
slurm = True

env_vars_list = []
for experiment_config in experiment_configs:
    for ds in ALL_DATASETS:
        ds_experiment_config = experiment_config.copy()
        if "DATASETS" in ds_experiment_config:
            raise ValueError("DATASETS should not be set in ds_experiment_config.")
        if "LABELED_DATASETS" in ds_experiment_config:
            raise ValueError(
                "LABELED_DATASETS should not be set in ds_experiment_config."
            )
        if "EVAL_DATASETS" in ds_experiment_config:
            raise ValueError("EVAL_DATASETS should not be set in ds_experiment_config.")

        ds_experiment_config["DATASETS"] = ds
        ds_experiment_config["LABELED_DATASETS"] = ds
        ds_experiment_config["EVAL_DATASETS"] = "burns"
        env_vars_list.append(make_env_vars_for_experiment(ds_experiment_config))

for env_vars in env_vars_list:
    print(make_train_command_for_experiment(env_vars, slurm=slurm), end="\n\n")

DATASETS=imdb EVAL_DATASETS=burns LABELED_DATASETS=imdb LAYER=-1 LR=0.01 METHOD_LIST=CCS+LR MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/normal-bananashed/layer_-1/ccs_lr/mode_concat/sup_weight_0/unsup_weight_1/lr_0.01/n_epochs_5000 NUM_SEEDS=1 N_EPOCHS=5000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTIONS=False SAVE_PARAMS=False SAVE_STATES=False SPAN_DIRS_COMBINATION=convex SUP_WEIGHT=0 TEST_PREFIX=normal-bananashed UNSUP_WEIGHT=1 WEIGHT_DECAY=0.0 sbatch --mem=16gb slurm_extract.sh

DATASETS=amazon-polarity EVAL_DATASETS=burns LABELED_DATASETS=amazon-polarity LAYER=-1 LR=0.01 METHOD_LIST=CCS+LR MODE=concat MODEL=meta-llama/Llama-2-13b-chat-hf NAME=llama-2-13b-chat-hf/normal-bananashed/layer_-1/ccs_lr/mode_concat/sup_weight_0/unsup_weight_1/lr_0.01/n_epochs_5000 NUM_SEEDS=1 N_EPOCHS=5000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTI

# CCS+LR

In [18]:
DEFAULT_CCS_LR_ENV_VARS = dict(
    METHOD_LIST="CCS+LR",
    MODE="concat",
    SUP_WEIGHT=10,
    UNSUP_WEIGHT=1,
    LR=3e-4,  # TODO: original is 1e-3
    N_EPOCHS=10000,
)

prefixes = ["normal-bananashed"]
models = [
    "meta-llama/Meta-Llama-3-8B-Instruct",
    # "meta-llama/Llama-2-13b-chat-hf",
    # "mistralai/Mistral-7B-Instruct-v0.2",
    # "meta-llama/Llama-2-7b-chat-hf",
]
layers = [-1]
weight_decay_list = [0, 1e-2, 1e-1, 1]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix, weight_decay in itertools.product(
    models, layers, prefixes, weight_decay_list
):
    experiment_configs.append(
        dict(
            DEFAULT_CCS_LR_ENV_VARS,
            MODEL=model,
            LAYER=layer,
            PREFIX=prefix,
            WEIGHT_DECAY=weight_decay,
        )
    )

print_train_commands_for_experiments_all_dataset_pairs(experiment_configs, slurm=True)

DATASETS=imdb EVAL_DATASETS="['imdb']" LABELED_DATASETS=imdb LAYER=-1 LR=0.0003 METHOD_LIST=CCS+LR MODE=concat MODEL=meta-llama/Meta-Llama-3-8B-Instruct NAME=meta-llama/Meta-Llama-3-8B-Instruct/normal-bananashed/layer_-1/ccs_lr/mode_concat/sup_weight_10/unsup_weight_1/lr_0.0003/n_epochs_10000 NUM_SEEDS=1 N_EPOCHS=10000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed SAVE_FIT_PLOTS=False SAVE_FIT_RESULT=True SAVE_ORTHOGONAL_DIRECTIONS=False SAVE_PARAMS=False SAVE_STATES=False SPAN_DIRS_COMBINATION=convex SUP_WEIGHT=10 TEST_PREFIX=normal-bananashed UNSUP_WEIGHT=1 WEIGHT_DECAY=0 sbatch  slurm_extract.sh

DATASETS=amazon-polarity EVAL_DATASETS="['imdb', 'amazon-polarity']" LABELED_DATASETS=imdb LAYER=-1 LR=0.0003 METHOD_LIST=CCS+LR MODE=concat MODEL=meta-llama/Meta-Llama-3-8B-Instruct NAME=meta-llama/Meta-Llama-3-8B-Instruct/normal-bananashed/layer_-1/ccs_lr/mode_concat/sup_weight_10/unsup_weight_1/lr_0.0003/n_epochs_10000 NUM_SEEDS=1 N_EPOCHS=10000 N_TRIES=1 OPT=sgd PREFIX=normal-bananashed SA

# CCS in LR span

## Train orthogonal probes (LR span)

In [ ]:
DEFAULT_LR_SPAN_ENV_VARS = dict(
    METHOD_LIST="CCS+LR-in-span",
    MODE="concat",
    SUP_WEIGHT=1,
    UNSUP_WEIGHT=0,
    LR=1e-2,
    N_EPOCHS=5000,
    NUM_ORTHOGONAL_DIRECTIONS=100,
    SPAN_DIRS_COMBINATION="convex",
)

prefixes = ["normal"]
models = [
    # "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Llama-2-13b-chat-hf",
    # "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
]
layers = [-1, -3, -5, -7, -9]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(
            DEFAULT_LR_SPAN_ENV_VARS,
            MODEL=model,
            LAYER=layer,
            PREFIX=prefix,
            SAVE_ORTHOGONAL_DIRECTIONS=True,
            SAVE_FIT_PLOTS=False,
        )
    )

# Make a train command for each individual dataset for each experiment config.
env_vars_list = []
for experiment_config in experiment_configs:
    for ds in ALL_DATASETS:
        ds_experiment_config = experiment_config.copy()
        if "DATASETS" in ds_experiment_config:
            raise ValueError("DATASETS should not be set in ds_experiment_config.")
        if "LABELED_DATASETS" in ds_experiment_config:
            raise ValueError(
                "LABELED_DATASETS should not be set in ds_experiment_config."
            )
        if "EVAL_DATASETS" in ds_experiment_config:
            raise ValueError("EVAL_DATASETS should not be set in ds_experiment_config.")

        ds_experiment_config["DATASETS"] = ds
        # LABELED_DATASETS is unused by LR, so this is arbitrary.
        ds_experiment_config["LABELED_DATASETS"] = ds
        # We don't care about the eval datasets because we're just generating
        # the LR span, but eval on all datasets just to have the results.
        ds_experiment_config["EVAL_DATASETS"] = "burns"
        env_vars_list.append(make_env_vars_for_experiment(ds_experiment_config))

slurm = True
for env_vars in env_vars_list:
    print(make_train_command_for_experiment(env_vars, slurm=slurm), end="\n\n")

## Train CCS in LR span

In [ ]:
DEFAULT_CCS_IN_LR_SPAN_ENV_VARS = dict(
    METHOD_LIST="CCS+LR-in-span",
    MODE="concat",
    SUP_WEIGHT=0,
    UNSUP_WEIGHT=1,
    LR=1e-2,
    N_EPOCHS=1000,
    SPAN_DIRS_COMBINATION="convex",
)

model_to_orthogonal_lr_directions_dir_template = {
    "meta-llama/Llama-2-13b-chat-hf": "/nas/ucb/ebronstein/Exhaustive-CCS/extraction_results/llama-2-13b-chat-hf/{prefix}/layer_{layer}/ccs_lr_in_span/mode_concat/sup_weight_1/unsup_weight_0/lr_0.01/n_epochs_5000/n_orth_dirs_100/span_dirs_combo_convex/meta-llama-Llama-2-13b-chat-hf",
    "mistralai/Mistral-7B-Instruct-v0.2": "/nas/ucb/ebronstein/Exhaustive-CCS/extraction_results/mistralai/Mistral-7B-Instruct-v0.2/{prefix}/layer_{layer}/ccs_lr_in_span/mode_concat/sup_weight_1/unsup_weight_0/lr_0.01/n_epochs_5000/n_orth_dirs_100/span_dirs_combo_convex/mistralai-Mistral-7B-Instruct-v0.2",
}


def get_orthogonal_lr_directions_dir(
    model: str, layer: int, prefix: str, dataset
) -> str:
    base_dir_template = model_to_orthogonal_lr_directions_dir_template[model]
    base_dir = base_dir_template.format(prefix=prefix, layer=layer)
    # Use the same dataset for the unlabeled and labeled datasets because the
    # LR span generation only used the unlabeled dataset.
    datasets_str = load_utils.get_combined_datasets_str(
        dataset, labeled_datasets=dataset
    )
    load_orthogonal_directions_dir = os.path.join(base_dir, datasets_str)
    if not os.path.exists(load_orthogonal_directions_dir):
        raise ValueError(
            f"Could not find orthogonal directions directory: {load_orthogonal_directions_dir}"
        )

    return load_orthogonal_directions_dir


prefixes = ["normal"]
models = [
    # "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Llama-2-13b-chat-hf",
    # "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
]
layers = [-1, -5, -9]
num_orth_dirs_list = [2, 4, 8, 16]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix, num_orth_dirs in itertools.product(
    models, layers, prefixes, num_orth_dirs_list
):
    for train_ds, test_ds in itertools.product(ALL_DATASETS, ALL_DATASETS):
        # Use the orthogonal LR directions from train_ds and train CCS in
        # their span on test_ds.
        load_orthogonal_directions_dir = get_orthogonal_lr_directions_dir(
            model, layer, prefix, train_ds
        )

        experiment_config = dict(
            DEFAULT_CCS_IN_LR_SPAN_ENV_VARS,
            MODEL=model,
            LAYER=layer,
            PREFIX=prefix,
            DATASETS=test_ds,
            LABELED_DATASETS=test_ds,  # Unused because the supervised weight is 0.
            EVAL_DATASETS=f'"{list(set([train_ds, test_ds]))}"',
            LOAD_ORTHOGONAL_DIRECTIONS_DIR=load_orthogonal_directions_dir,
            NUM_ORTHOGONAL_DIRECTIONS=num_orth_dirs,
        )
        env_vars = make_env_vars_for_experiment(experiment_config)
        print(make_train_command_for_experiment(env_vars, slurm=True), end="\n\n")

# LR in CCS span

## Train orthogonal probes (CCS span)

In [ ]:
DEFAULT_CCS_SPAN_ENV_VARS = dict(
    METHOD_LIST="CCS+LR-in-span",
    MODE="concat",
    SUP_WEIGHT=0,
    UNSUP_WEIGHT=1,
    LR=1e-2,
    N_EPOCHS=1000,
    NUM_ORTHOGONAL_DIRECTIONS=100,
    SPAN_DIRS_COMBINATION="convex",
)

prefixes = ["normal"]
models = [
    # "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Llama-2-13b-chat-hf",
    # "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
]
layers = [-1, -3, -5, -7, -9]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix in itertools.product(models, layers, prefixes):
    experiment_configs.append(
        dict(
            DEFAULT_CCS_SPAN_ENV_VARS,
            MODEL=model,
            LAYER=layer,
            PREFIX=prefix,
            SAVE_ORTHOGONAL_DIRECTIONS=True,
            SAVE_FIT_PLOTS=False,
        )
    )

# Make a train command for each individual dataset for each experiment config.
env_vars_list = []
for experiment_config in experiment_configs:
    for ds in ALL_DATASETS:
        ds_experiment_config = experiment_config.copy()
        if "DATASETS" in ds_experiment_config:
            raise ValueError("DATASETS should not be set in ds_experiment_config.")
        if "LABELED_DATASETS" in ds_experiment_config:
            raise ValueError(
                "LABELED_DATASETS should not be set in ds_experiment_config."
            )
        if "EVAL_DATASETS" in ds_experiment_config:
            raise ValueError("EVAL_DATASETS should not be set in ds_experiment_config.")

        ds_experiment_config["DATASETS"] = ds
        # LABELED_DATASETS is unused by CCS, so this is arbitrary.
        ds_experiment_config["LABELED_DATASETS"] = ds
        # We don't care about the eval datasets because we're just generating
        # the LR span, but eval on all datasets just to have the results.
        ds_experiment_config["EVAL_DATASETS"] = "burns"
        env_vars_list.append(make_env_vars_for_experiment(ds_experiment_config))

slurm = True
for env_vars in env_vars_list:
    print(make_train_command_for_experiment(env_vars, slurm=slurm), end="\n\n")

## Train LR in CCS span

In [ ]:
DEFAULT_LR_IN_CCS_SPAN_ENV_VARS = dict(
    METHOD_LIST="CCS+LR-in-span",
    MODE="concat",
    SUP_WEIGHT=1,
    UNSUP_WEIGHT=0,
    LR=1e-2,
    N_EPOCHS=5000,
    SPAN_DIRS_COMBINATION="convex",
)

model_to_orthogonal_ccs_directions_dir_template = {
    "meta-llama/Llama-2-13b-chat-hf": "/nas/ucb/ebronstein/Exhaustive-CCS/extraction_results/llama-2-13b-chat-hf/{prefix}/layer_{layer}/ccs_lr_in_span/mode_concat/sup_weight_0/unsup_weight_1/lr_0.01/n_epochs_1000/n_orth_dirs_100/span_dirs_combo_convex/meta-llama-Llama-2-13b-chat-hf",
    "mistralai/Mistral-7B-Instruct-v0.2": "/nas/ucb/ebronstein/Exhaustive-CCS/extraction_results/mistralai/Mistral-7B-Instruct-v0.2/{prefix}/layer_{layer}/ccs_lr_in_span/mode_concat/sup_weight_0/unsup_weight_1/lr_0.01/n_epochs_1000/n_orth_dirs_100/span_dirs_combo_convex/mistralai-Mistral-7B-Instruct-v0.2",
}


def get_orthogonal_ccs_directions_dir(
    model: str, layer: int, prefix: str, dataset
) -> str:
    base_dir_template = model_to_orthogonal_ccs_directions_dir_template[model]
    base_dir = base_dir_template.format(prefix=prefix, layer=layer)
    # Use the same dataset for the unlabeled and labeled datasets because the
    # CCS span generation only used the unlabeled dataset.
    datasets_str = load_utils.get_combined_datasets_str(
        dataset, labeled_datasets=dataset
    )
    load_orthogonal_directions_dir = os.path.join(base_dir, datasets_str)
    if not os.path.exists(load_orthogonal_directions_dir):
        raise ValueError(
            f"Could not find orthogonal directions directory: {load_orthogonal_directions_dir}"
        )

    return load_orthogonal_directions_dir


prefixes = ["normal"]
models = [
    # "meta-llama/Llama-2-7b-chat-hf",
    "meta-llama/Llama-2-13b-chat-hf",
    # "meta-llama/Meta-Llama-3-8B-Instruct",
    "mistralai/Mistral-7B-Instruct-v0.2",
]
layers = [-1, -5, -9]
num_orth_dirs_list = [2, 4, 8, 16]

# Iterate over product of parameters.
experiment_configs = []
for model, layer, prefix, num_orth_dirs in itertools.product(
    models, layers, prefixes, num_orth_dirs_list
):
    for train_ds, test_ds in itertools.product(ALL_DATASETS, ALL_DATASETS):
        # Use the orthogonal CCS directions from test_ds and train LR in
        # their span on train_ds.
        load_orthogonal_directions_dir = get_orthogonal_ccs_directions_dir(
            model, layer, prefix, test_ds
        )

        experiment_config = dict(
            DEFAULT_LR_IN_CCS_SPAN_ENV_VARS,
            MODEL=model,
            LAYER=layer,
            PREFIX=prefix,
            DATASETS=train_ds,  # Unused because the unsupervised weight is 0.
            LABELED_DATASETS=train_ds,
            EVAL_DATASETS=f'"{list(set([train_ds, test_ds]))}"',
            LOAD_ORTHOGONAL_DIRECTIONS_DIR=load_orthogonal_directions_dir,
            NUM_ORTHOGONAL_DIRECTIONS=num_orth_dirs,
        )
        env_vars = make_env_vars_for_experiment(experiment_config)
        print(make_train_command_for_experiment(env_vars, slurm=True), end="\n\n")